In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

*** Earth Engine *** FINAL DEADLINE: ee.Authenticate will fail after 2022-06-06. Please upgrade. https://developers.google.com/earth-engine/guides/python_install


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
114.1917, 30.3742,114.2449, 30.4138

In [ ]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
# # 114.5713, 30.6201, 114.6410, 30.6889
# # 113.7452, 30.1890,113.7776, 30.2260
# # 114.4544, 30.7685,114.5423, 30.8368
# q = ee.Geometry.Rectangle([114.5713, 30.6201, 114.6410, 30.6889])
# Map.addLayer(q, {}, "q")
# Map.centerObject(q,14)

In [ ]:
region = 'wuhan'
elevation = ee.Image("users/311605001111/hillshade_" + region)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

# 单景

In [ ]:
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi)        
# l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
#        .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
#        .filterBounds(roi) \
#        .filter(ee.Filter.lt('CLOUD_COVER',80))
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi)
l58 = l8.merge(l5)
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}

year = 2000
startDate = str(year) + '-01-01'
endDate = str(year) + '-12-31'
refer_img = l58.filterDate(startDate, endDate).map(maskSR).median().clip(roi)
Map.addLayer(refer_img, visParams,'refer_img')

basemap = ee.Image.constant(0).clip(roi).rename('waterclass')

result = ee.Image('users/311605001111/WF/' + region + '_WF_' + str(year)).rename('waterclass')
permanent = result.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
season= ee.ImageCollection([result.gte(0.25),result.lt(0.75)]).sum().eq(2).rename('waterclass')
pw = ee.ImageCollection([permanent,season,basemap]).sum()
Map.addLayer(pw.selfMask(),{'palette':['#87CEFA', 'blue'],'min':1,'max':2},"pw")

pw_no = ee.Image('users/311605001111/WF_nocorrect/' + region + '_WF_' + str(year)).gte(0.75)
Map.addLayer(pw_no.selfMask(),{'palette':['purple']},"pw_no")


# JRC
JRC = ee.Image('JRC/GSW1_3/YearlyHistory/' + str(year)).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
jrc_waterclass = ee.ImageCollection([JRC,basemap]).sum()
Map.addLayer(jrc_waterclass.selfMask(),{'palette':['#bfd69b', 'green'],'min':1,'max':2},"jrc")

# Maryland
Maryland = ee.Image('users/311605001111/Maryland/Maryland_nationwide_' + str(year)).clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
Map.addLayer(maryland_waterclass.selfMask(),{'palette':['#4ad6c3','red'],'min':1,'max':2},"maryland")


In [ ]:
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi)        
# l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
#        .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
#        .filterBounds(roi) \
#        .filter(ee.Filter.lt('CLOUD_COVER',80))
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi)
l58 = l8.merge(l5)
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}

year = 1999
startDate = str(year) + '-01-01'
endDate = str(year) + '-12-31'
refer_img = l58.filterDate(startDate, endDate).map(maskSR).median().clip(roi)
Map.addLayer(refer_img, visParams,'refer_img')

pw = ee.Image('users/311605001111/WF/wuhan_WF_' + str(year)).gte(0.75)
Map.addLayer(pw.selfMask(),{'palette':['blue']},"pw")

# pw_no = ee.Image('users/311605001111/WF_nocorrect/wuhan_WF_' + str(year)).gte(0.75)
# Map.addLayer(pw_no.selfMask(),{'palette':['purple']},"pw_no")

jrc = ee.Image('JRC/GSW1_3/YearlyHistory/' + str(year)).gte(3).clip(roi)
Map.addLayer(jrc.selfMask(),{'palette':['green']},"jrc")

# maryland = ee.Image('users/311605001111/Maryland/Maryland_nationwide_' + str(year)).clip(roi).gte(75)
# Map.addLayer(maryland.selfMask(),{'palette':['red']},"maryland")

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee

year = 1999
days_31 = [1,3,5,7,8,10,12]
days_30 = [4,6,9,11]
days_28 = [2]
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
region = [113.8638, 30.5915, 114.4089, 30.9231]
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa'])
for i in range(1,13,1):
    if i in days_31:
        images = l5.filterDate('{}-{}-01'.format(year,i),'{}-{}-31'.format(year,i)).map(maskSR).median().clip(roi)
    elif i in days_30:
        images = l5.filterDate('{}-{}-01'.format(year,i),'{}-{}-30'.format(year,i)).map(maskSR).median().clip(roi)
    else:
        images = l5.filterDate('{}-{}-01'.format(year,i),'{}-{}-28'.format(year,i)).map(maskSR).median().clip(roi)
    refer_img = images.visualize(**visParams)
    fig = plt.figure(figsize=(12, 8))
    ax = cartoee.get_map(refer_img, region=region)
    cartoee.add_gridlines(ax, interval=[0.1,0.1], linestyle=":")
    label = 'Experimental result ('+ str(i) + ")"
    ax.set_title(label = label, fontsize=28)

# 对比区域

## 区域一

In [ ]:
# 114.5713, 30.6201, 114.6410, 30.6889
# 113.7452, 30.1890,113.7776, 30.2260
# 114.4544, 30.7685,114.5423, 30.8368

In [ ]:
# 区域一
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
roi = ee.Geometry.Rectangle([114.5713, 30.6201, 114.6410, 30.6889])
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi)        
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi)
l58 = l8.merge(l5)
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
region = [114.5713, 30.6201, 114.6410, 30.6889]


for year in range(2000,2021,5):
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    refer_img = l58.filterDate(startDate, endDate).map(maskSR).median().clip(roi).visualize(**visParams)
    # pw = ee.Image('JRC/GSW1_3/YearlyHistory/' + str(year)).gte(3).clip(roi)
    # label = 'GSWD ('+ str(year) + ")"
    # pw = ee.Image('users/311605001111/WF/wuhan_WF_' + str(year)).gte(0.75)
    # label = 'Experimental result ('+ str(year) + ")"
    pw = ee.Image('users/311605001111/Maryland/Maryland_nationwide_' + str(year)).clip(roi).gte(75)
    label = 'GLAD ('+ str(year) + ")"
    blend = refer_img.blend(pw.selfMask().visualize(**{'palette':['blue']}))
    fig = plt.figure(figsize=(12, 8))
    ax = cartoee.get_map(blend, region=region)
    cartoee.add_gridlines(ax, interval=[0.03,0.03], linestyle=":")
    ax.set_title(label = label, fontsize=28)

In [ ]:
# 区域一
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
roi = ee.Geometry.Rectangle([114.5713, 30.6201, 114.6410, 30.6889])
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi)        
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi)
l58 = l8.merge(l5)
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
region = [114.5713, 30.6201, 114.6410, 30.6889]


for year in range(2000,2021,5):
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    refer_img = l58.filterDate(startDate, endDate).map(maskSR).median().clip(roi).visualize(**visParams)
    # pw = ee.Image('JRC/GSW1_3/YearlyHistory/' + str(year)).gte(3).clip(roi)
    pw = ee.Image('users/311605001111/WF/wuhan_WF_' + str(year)).gte(0.75)
    # pw = ee.Image('users/311605001111/Maryland/Maryland_nationwide_' + str(year)).clip(roi).gte(75)
    blend = refer_img.blend(pw.selfMask().visualize(**{'palette':['blue']}))
    fig = plt.figure(figsize=(12, 8))
    ax = cartoee.get_map(blend, region=region)
    # cartoee.add_gridlines(ax, interval=[0.1,0.1], linestyle=":")
    label = 'Experimental result ('+ str(year) + ")"
    ax.set_title(label = label, fontsize=28)

##  区域二

In [ ]:
113.7452, 30.1890,113.7776, 30.2260

In [ ]:
# 区域二
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
roi = ee.Geometry.Rectangle([113.7452, 30.1890,113.7776, 30.2260])
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi)        
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi)
l58 = l8.merge(l5)
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
region = [113.7452, 30.1890,113.7776, 30.2260]


for year in range(2000,2021,5):
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    refer_img = l58.filterDate(startDate, endDate).map(maskSR).median().clip(roi).visualize(**visParams)
    # pw = ee.Image('JRC/GSW1_3/YearlyHistory/' + str(year)).gte(3).clip(roi)
    # label = 'GSWD ('+ str(year) + ")"
    pw = ee.Image('users/311605001111/WF/wuhan_WF_' + str(year)).gte(0.75)
    label = 'Experimental result ('+ str(year) + ")"
    # pw = ee.Image('users/311605001111/Maryland/Maryland_nationwide_' + str(year)).clip(roi).gte(75)
    # label = 'GLAD ('+ str(year) + ")"
    blend = refer_img.blend(pw.selfMask().visualize(**{'palette':['blue']}))
    fig = plt.figure(figsize=(12, 8))
    ax = cartoee.get_map(blend, region=region)
    cartoee.add_gridlines(ax, interval=[0.02,0.02], linestyle=":")
    ax.set_title(label = label, fontsize=28)